You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	JOIN Operation:
o	Perform a "JOIN" operation to combine information from the launches data and the rockets data.
o	Join Logic: For each launch record, look up the corresponding rocket record using the rocket identifier. The join key is the rocket.rocket_id field within each launch record, which matches the rocket_id field in the rocket records. Add the rocket_name from the matched rocket record to the launch record. (This is conceptually similar to a left join, keeping all launch records).
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with the corresponding rocket_name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable: 
	LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
	ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval (for launches, for rockets), number of launches retrieved, number of rockets retrieved, starting join operation, join operation complete, number of records after join, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + join operation). Print this time after the join operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting joined list of dictionaries from the join operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting joined JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
# Databricks Notebook
# SpaceX Launches + Rockets Join and Upload
# Author: ChatGPT
# Date: 2025-04-09

# COMMAND ----------

# Import required modules
import requests
import logging
import time
import json

# COMMAND ----------

# Control parameters
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
UPLOAD_URL = "https://httpbin.org/post"

# COMMAND ----------

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

# COMMAND ----------

# Start total pipeline timer
pipeline_start_time = time.time()

# COMMAND ----------

def fetch_api_data(url):
    """Fetch data from an API endpoint with error handling."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises an HTTPError if the response was an HTTP error
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching data from {url}: {e}")
        raise

# COMMAND ----------

# Start main operation timer
main_ops_start_time = time.time()

# Fetch launches
logger.info("Starting data retrieval: launches")
launches_data = fetch_api_data(LAUNCHES_API_URL)
logger.info(f"Number of launches retrieved: {len(launches_data)}")

# Fetch rockets
logger.info("Starting data retrieval: rockets")
rockets_data = fetch_api_data(ROCKETS_API_URL)
logger.info(f"Number of rockets retrieved: {len(rockets_data)}")

# COMMAND ----------

# Create a lookup dictionary for rockets by rocket_id
rocket_lookup = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets_data}

# COMMAND ----------

# Perform the join operation
logger.info("Starting join operation between launches and rockets")

enriched_launches = []
for launch in launches_data:
    rocket_id = launch.get('rocket', {}).get('rocket_id')
    rocket_name = rocket_lookup.get(rocket_id, "Unknown Rocket")
    enriched_launch = launch.copy()
    enriched_launch['rocket_name'] = rocket_name
    enriched_launches.append(enriched_launch)

logger.info("Join operation complete")
logger.info(f"Number of enriched launch records: {len(enriched_launches)}")

# COMMAND ----------

# End main operation timer and print duration
main_ops_end_time = time.time()
main_ops_duration = main_ops_end_time - main_ops_start_time
logger.info(f"Main operations (data retrieval + join) completed in {main_ops_duration:.2f} seconds")

# COMMAND ----------

# Serialize to JSON
enriched_launches_json = json.dumps(enriched_launches)

# COMMAND ----------

# Upload to httpbin.org
logger.info("Starting upload to httpbin.org")

try:
    upload_response = requests.post(UPLOAD_URL, headers={"Content-Type": "application/json"}, data=enriched_launches_json, timeout=10)
    upload_response.raise_for_status()
    logger.info(f"Upload successful. Status Code: {upload_response.status_code}")
except requests.exceptions.RequestException as e:
    logger.error(f"Error during upload to {UPLOAD_URL}: {e}")
    raise

# COMMAND ----------

# End total pipeline timer and print total execution time
pipeline_end_time = time.time()
total_pipeline_duration = pipeline_end_time - pipeline_start_time
logger.info(f"Total pipeline execution time: {total_pipeline_duration:.2f} seconds")
